## Loading the dependencies for the Recommendation system implementation

In [15]:
## Load dependencies for the Recommendation System
import os
from dotenv import load_dotenv
import cohere
from langchain.vectorstores import FAISS
from langchain_core.embeddings import Embeddings

# Load environment variables
load_dotenv()
co = cohere.Client(os.getenv("COHERE_API_KEY"))


## Defining an embedding wrapper

In [16]:
## Define embedding wrapper with graceful fallback
class CohereEmbedder(Embeddings):
    def embed_documents(self, texts):
        try:
            return co.embed(
                texts=texts,
                model="embed-english-v3.0",
                input_type="search_document"
            ).embeddings
        except Exception as e:
            print(f" Embedding failed: {e}")
            return [[0.0] * 1024] * len(texts)

    def embed_query(self, text):
        return self.embed_documents([text])[0]

embedding_model = CohereEmbedder()


## Load the FAISS Index


In [18]:
vector_store = FAISS.load_local(
    folder_path="vectorstore/faiss_large",
    index_name="large_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


## Recomender function 

In [19]:
## Define Recommendation Function
def recommend_related_papers(query_text, top_k=5):
    results = vector_store.similarity_search(query_text, k=top_k)
    for i, doc in enumerate(results, 1):
        print(f"--- Recommendation #{i} ---\n{doc.page_content[:500]}...\n")


In [ ]:
query = "Shaping the globular cluster mass function by stellar-dynamical evaporation"
recommend_related_papers(query, top_k=5)
